In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [ ]:
def find_warsaw_district(input_string) -> str:
    """
    Identifies and returns a Warsaw district name from a given input string.

    The function splits the input string by commas and checks each part against a list of Warsaw district names. 
    If one of the parts matches a district name, that district name is returned. If there are no matches, 
    the function returns None.

    Parameters:
    - input_string (str): The string to be analyzed, which can contain multiple comma-separated values.

    Returns:
    - str or None: The name of a Warsaw district if found; otherwise, None.
    """
    # List of Warsaw districts
    warsaw_districts = [
        "Bemowo", "Białołęka", "Bielany", "Mokotów", "Ochota",
        "Praga-Południe", "Praga-Północ", "Rembertów", "Śródmieście",
        "Targówek", "Ursus", "Ursynów", "Wawer", "Wesoła",
        "Wilanów", "Włochy", "Wola", "Żoliborz"
    ]

    # Split the input string by commas
    parts = input_string.split(',')

    # Check each part for a match with Warsaw districts
    for part in parts:
        if part.strip() in warsaw_districts:
            return part.strip()  # Return the matching district name

    return None

In [ ]:
def extract_ad_dates(row):
    ad_info = row['announcement_date']
    
    update_date_part = [line for line in ad_info.split('\\n') if 'Aktualizacja:' in line]
    last_update = update_date_part[0].replace("('Aktualizacja: ", '').strip() if update_date_part else None
    
    added_date_part = [line for line in ad_info.split('\\n') if 'Dodano:' in line]
    ad_added = added_date_part[0].replace('Dodano: ', '').strip() if added_date_part else None
    
    return pd.Series([last_update, ad_added])

In [ ]:
def process_data(data_draw):
    
    df = data_draw.copy()
    df['district'] = df['location'].apply(lambda x: find_warsaw_district(str(x)))
    df = df[~df.district.isna()]
    
    df[['last_update', 'added_dt']] = df.apply(extract_ad_dates, axis=1)
    df.drop(['announcement_date'], axis=1, inplace=True)
    df['expired'] = 0
    df['expired_date'] = None
    to_order = ['added_dt', 'last_update', 'link', 'expired', 'expired_date']
    columns_order = to_order + [col for col in df.columns if col not in to_order]
    df = df[columns_order]
    
    return df

In [ ]:
df = pd.read_csv('data_raw\otodom_scraped_data\otodom_2025_01_05.csv')

In [ ]:
df = process_data(df)

In [2]:
import pandas as pd

In [4]:
path = f'data_processed/main.csv'
main = pd.read_csv(path)

In [5]:
sorted([date for date in list(main.expired_date.unique()) if isinstance(date, str)])

['2024_12_29',
 '2025_01_02',
 '2025_01_03',
 '2025_01_04',
 '2025_01_05',
 '2025_01_06',
 '2025_01_07',
 '2025_01_08',
 '2025_01_09',
 '2025_01_10',
 '2025_01_11',
 '2025_01_12',
 '2025_01_13',
 '2025_01_14',
 '2025_01_15',
 '2025_01_16',
 '2025_01_17',
 '2025_01_18',
 '2025_01_19',
 '2025_01_20',
 '2025_01_21',
 '2025_01_22',
 '2025_01_23',
 '2025_01_24',
 '2025_01_25',
 '2025_01_26',
 '2025_01_27',
 '2025_01_28',
 '2025_01_29',
 '2025_01_30',
 '2025_01_31',
 '2025_02_01',
 '2025_02_02',
 '2025_02_03',
 '2025_02_04',
 '2025_02_06',
 '2025_02_08',
 '2025_02_09',
 '2025_02_12',
 '2025_02_13',
 '2025_02_15',
 '2025_02_16',
 '2025_02_19',
 '2025_02_20',
 '2025_02_21',
 '2025_02_22',
 '2025_02_23',
 '2025_02_24',
 '2025_02_25',
 '2025_02_27']

In [8]:
main['expired_date'] = main['expired_date'].replace('2025_02_27', '2025_02_26')

In [10]:
sorted([date for date in list(main.expired_date.unique()) if isinstance(date, str)])

['2024_12_29',
 '2025_01_02',
 '2025_01_03',
 '2025_01_04',
 '2025_01_05',
 '2025_01_06',
 '2025_01_07',
 '2025_01_08',
 '2025_01_09',
 '2025_01_10',
 '2025_01_11',
 '2025_01_12',
 '2025_01_13',
 '2025_01_14',
 '2025_01_15',
 '2025_01_16',
 '2025_01_17',
 '2025_01_18',
 '2025_01_19',
 '2025_01_20',
 '2025_01_21',
 '2025_01_22',
 '2025_01_23',
 '2025_01_24',
 '2025_01_25',
 '2025_01_26',
 '2025_01_27',
 '2025_01_28',
 '2025_01_29',
 '2025_01_30',
 '2025_01_31',
 '2025_02_01',
 '2025_02_02',
 '2025_02_03',
 '2025_02_04',
 '2025_02_06',
 '2025_02_08',
 '2025_02_09',
 '2025_02_12',
 '2025_02_13',
 '2025_02_15',
 '2025_02_16',
 '2025_02_19',
 '2025_02_20',
 '2025_02_21',
 '2025_02_22',
 '2025_02_23',
 '2025_02_24',
 '2025_02_25',
 '2025_02_26']

In [ ]:
main = pd.concat([main, df], ignore_index=True)

In [12]:
path = f'data_processed/main.csv'
main.to_csv(path,
          encoding='utf-8',
          index=False)

### ====

check for expired date

In [14]:
import pandas as pd
path = f'data_processed/main.csv'
main = pd.read_csv(path)

In [19]:
date = '2025_02_10'
df = pd.read_csv(f'data_raw/search_for_inactive/search_for_inactive_{date}.csv')

df = df[df.expired.eq(True)]
check = pd.merge(df, main[['link', 'expired_date', 'district']], how='left', on='link')

check['check'] = [True if a==b else False for a,b in zip(check['expired_date_x'], check['expired_date_y'])]
check[check.check.eq(False)]

,link,expired,expired_date_x,expired_date_y,district,check


In [17]:
main.update(main[['link']].merge(df, on='link', how='left'), overwrite=True)

In [ ]:
date2 = '2025_01_25'
df2 = pd.read_csv(f'data_raw/search_for_inactive/search_for_inactive_{date2}.csv')
df2 = df2[df2.expired.eq(True)]

In [ ]:
df = check = pd.merge(df1, df2, how='outer', suffixes=('_24', '_25'), on='link')

In [ ]:
date1 = '2025_01_05'
df1 = pd.read_csv(f'data_raw/search_for_inactive/search_for_inactive_{date1}.csv')
df1 = df1[df1.expired.eq(True)]

date = '2025_01_06'
path = f'data_raw/search_for_inactive/search_for_inactive_{date}.csv'
df = pd.read_csv(path)

In [ ]:
df = df[~df.link.isin(df1.link)]
df[df.expired.eq(True)]

In [ ]:
df.to_csv(path,
          encoding='utf-8',
          index=False)

### ====

fill date added

In [ ]:
new_records = process_data(new_records)
new_records = new_records[~new_records.link.isin(main.link)]

# Concat main and newly scraped announcements
main = pd.concat([main, new_records], ignore_index=True)


In [325]:
import pandas as pd
path = f'data_processed/main.csv'
main = pd.read_csv(path)

In [12]:
main.head(1)

,added_dt,last_update,link,expired,expired_date,title,rent_price,area_room_num,floor,ogrzewanie,...,approximate_coordinates,year_of_construction,elevator,building_type,security,equipment,utilities,safeguards,adv_description,district
0,22.12.2024,22.12.2024,https://www.otodom.pl/pl/oferta/piekne-mieszka...,1,2025_01_04,Piękne mieszkanie na warszawskich Skoroszach,3 000 zł\n/miesiąc\n+ Czynsz 600 zł,45m²\nWynajmę również studentom\n2 pokoje,3/6,inne,...,False,2013.0,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,NaN,drzwi / okna antywłamaniowe\ndomofon / wideofon,Wynajmę mieszkanie 45 metrów na warszawskim Ur...,Ursus


In [337]:
date = '2025_01_31'
df = pd.read_csv(f'data_raw/otodom_scraped_data/otodom_{date}.csv')

In [339]:
from toolkit import *

In [343]:
new_records = process_data(df)
new_records = new_records[~new_records.link.isin(main.link)]

# Concat main and newly scraped announcements
main = pd.concat([main, new_records], ignore_index=True)

In [345]:
main.loc[main.link.isin(df.link), 'added_dt'] = date

In [347]:
main[main.link.isin(df.link)].added_dt.unique()

array(['2025_01_31'], dtype=object)

In [349]:
path = f'data_processed/main.csv'
main.to_csv(path,
          encoding='utf-8',
          index=False)